<a href="https://colab.research.google.com/github/BobZhang26/AIPI-590---XAI-Assignment-3/blob/main/XAI_assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment # 3
### Description
### Bob Zhang

#### Click the badge below to direct to GitHub host:

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/BobZhang26/AIPI-590---XAI-Assignment-3/blob/main/XAI_assignment_3.ipynb)

In [9]:
# Please use this to connect your GitHub repository to your Google Colab notebook
# Connects to any needed files from GitHub and Google Drive
import os

# Remove Colab default sample_data
!rm -r ./sample_data
!rm -rf AIPI-590---XAI-Assignment-3
!rm -rf AIPI-590---XAI-Assignment-3 2>/dev/null


# Clone GitHub files to colab workspace
repo_name = "AIPI-590---XAI-Assignment-3" # Change to your repo name
git_path = 'https://github.com/BobZhang26/AIPI-590---XAI-Assignment-3.git' #Change to your path
!git clone "{git_path}"

# Install dependencies from requirements.txt file
#!pip install -r "{os.path.join(repo_name,'requirements.txt')}" #Add if using requirements.txt

# Change working directory to location of notebook
notebook_dir = "/content/AIPI-590---XAI-Assignment-3"
path_to_notebook = os.path.join(repo_name,notebook_dir)
%cd "{notebook_dir}"
# %ls
# print cwd
print(os.getcwd())
print(path_to_notebook)

# change working directory to notebook




rm: cannot remove './sample_data': No such file or directory
Cloning into 'AIPI-590---XAI-Assignment-3'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 143 (delta 71), reused 12 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (143/143), 1.32 MiB | 8.83 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/AIPI-590---XAI-Assignment-3
/content/AIPI-590---XAI-Assignment-3
/content/AIPI-590---XAI-Assignment-3


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

data = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")

data.head()

<ipython-input-8-398046bf4d1b>:6: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Task 1: Exploratory Analysis

### 1.1 Data Overview and Cleaning
Data contains 7043 observations with 20 independent variables and 1 dependent variable (Churn). There are apparently 3 numerical columns `SeniorCitizen`,`tenure`, and `MonthlyCharges`; the rest are categorical columns. However, `SeniorCitizen` column is a binary feature where 1 indicates senior citizens and 0 indicates not senior citizens. This should be corrected from numerical to categorical.
There is no missing value in the dataset.

In [ ]:
# load the data and check each column's type: categorical or numerical
print("Check column types:")
print(data.info())
print()
print("Check column types again:")
print(data.info())
print()
# check missing values or anormalies
print("Check missing values:")
print(data.isnull().sum())
print()
# summary statistics of the data
print("Summary statistics of the data:")
print(data.describe())

The numerical columns are `tenure`, `MonthlyCharges` and `TotalCharges`. The rest are all categorical. Correct them from the dataframe

In [ ]:
cat_cols = data.columns.difference(['tenure', 'MonthlyCharges', 'TotalCharges'])
data[cat_cols] = data[cat_cols].astype('category')

# for numerical columns
data['tenure'] = data['tenure'].astype('int64')
data['MonthlyCharges'] = data['MonthlyCharges'].astype('float64')
data['TotalCharges'] = data['TotalCharges'].replace(" ", np.nan).astype('float64')

# drop the rows where TotalCharges is nan, the cleaned dataframe is df_clean
df = data.copy()
df_clean = df.dropna()
df_clean.info()

### 1.2 Univariate Analysis
Target Variable (Churn) Distribution: Visualize the distribution of the target variable (churn) to understand class imbalance. Use a bar chart to visualize the churn rate.



In [ ]:
sns.countplot(x='Churn', data=df_clean)
plt.xlabel('Churn')
plt.ylabel('Count')
plt.title('Churn Distribution')
plt.show()

Now we are looking at the distributions of numerical variables: `tenure`and `MonthlyCharges`. This will help identify whether variables are normally distributed as to meet `linear regression assumption`.

In [ ]:
# Distribution of tenure, MonthlyCharges and TotalCharges
num_col = ["tenure", "MonthlyCharges","TotalCharges"]
plt.figure(figsize=(15,4))
for i,var in enumerate(num_col):
  plt.subplot(1,3,i+1)
  sns.histplot(df_clean[var], kde=True)
  plt.title(f"Distribution of {var}")
  plt.xlabel(var)
  plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of categorical variables
cat_colss = cat_cols.difference(['customerID','Churn'])
plt.figure(figsize=(25,15))
for i ,var in enumerate(cat_colss):
  plt.subplot(4,4,i+1)
  sns.countplot(x='Churn',data=df_clean, hue = var)
  plt.title(f"Distribution of {var}")
plt.tight_layout()
plt.show()